<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Разделение-выборок" data-toc-modified-id="Разделение-выборок-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Разделение выборок</a></span></li><li><span><a href="#Используем-технологию-TF-IDF-для-обработки-данныx,-а-так-же-отфильтруем-содержимое-при-помощи-библиотеки-nltk" data-toc-modified-id="Используем-технологию-TF-IDF-для-обработки-данныx,-а-так-же-отфильтруем-содержимое-при-помощи-библиотеки-nltk-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Используем технологию TF-IDF для обработки данныx, а так же отфильтруем содержимое при помощи библиотеки nltk</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-регрессия" data-toc-modified-id="Логистическая-регрессия-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая регрессия</a></span></li><li><span><a href="#Модель-дерево-решений" data-toc-modified-id="Модель-дерево-решений-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Модель дерево решений</a></span></li><li><span><a href="#Модель-случайный-лес" data-toc-modified-id="Модель-случайный-лес-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Модель случайный лес</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.metrics import confusion_matrix
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import make_scorer
from sklearn.utils import shuffle
import time
from sklearn.feature_extraction.text import CountVectorizer 
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer 
from pymystem3 import Mystem
from nltk.stem import WordNetLemmatizer
import re 
from sklearn.metrics import classification_report

from sklearn.metrics import precision_recall_curve
from sklearn.metrics import precision_score, recall_score
from sklearn.metrics import roc_curve 
from sklearn.metrics import roc_auc_score
import plotly.graph_objects as go
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [2]:
df=pd.read_csv('/datasets/toxic_comments.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
df.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
df.shape

(159292, 3)

In [6]:
# удалим ненужный столбец
df.drop(columns = ['Unnamed: 0'],axis = 1)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0
...,...,...
159287,""":::::And for the second time of asking, when ...",0
159288,You should be ashamed of yourself \n\nThat is ...,0
159289,"Spitzer \n\nUmm, theres no actual article for ...",0
159290,And it looks like it was actually you who put ...,0


In [7]:
# проверим процент токсичных комментариев:
df['toxic'].value_counts()/df.shape[0]*100

0    89.838787
1    10.161213
Name: toxic, dtype: float64

In [8]:
# проверим на дубликаты
df.duplicated().mean()

0.0

**ВЫВОД:**

- В датасете 159292 строки. 
- Дубликатов и пропусков нет. 
- Оставили 2 столбца: toxic, text. 
- В столбце text содержатся тексты твитов. 
- В столбе toxic булевые значения является ли данный твит токсичным или нет. Оценить тональность — значит отметить текст как позитивный или негативный. То есть мы решаем задачу классификации, где целевой признак равен «1» для положительного текста и «0» для отрицательного. Заметен довольно большой дисбаланс классов в сторону отрицательных комментариев. 89.8 % твитов токсичны.



In [9]:
# Объявим корпус текстов
corpus = list(df['text'])
print(corpus[0])

Explanation
Why the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I voted at New York Dolls FAC. And please don't remove the template from the talk page since I'm retired now.89.205.38.27


In [10]:
# Объявим лемматизатор и функцию для очистки и лемматизации текстов "clear_text
lemmatizer = WordNetLemmatizer()
def clear_text(text):
    y=re.sub(r"[^'a-zA-Z ]", ' ', text) 
    k=" ".join(y.split())
    return k

In [11]:
# создадим функцию которая будет использовать функцию по очистке слов и лемматизировать
from nltk.corpus import wordnet
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

In [12]:
def lemmafunction(textc):
    k=[]
    for i in nltk.word_tokenize(textc):
        y=lemmatizer.lemmatize(i, get_wordnet_pos(i))
        k.append(y)
    return ' '.join(k) 

lemy=[]
for i in tqdm(range(len(corpus))):
    
    lemy.append(lemmafunction(clear_text(corpus[i])))
df['lemm_text']=pd.Series(lemy, index=df.index)

  0%|          | 0/159292 [00:00<?, ?it/s]

In [13]:
# проверка текст до
df['text'][3]

'"\nMore\nI can\'t make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of ""types of accidents""  -I think the references may need tidying so that they are all in the exact same format ie date format etc. I can do that later on, if no-one else does first - if you have any preferences for formatting style on references or want to do it yourself please let me know.\n\nThere appears to be a backlog on articles for review so I guess there may be a delay until a reviewer turns up. It\'s listed in the relevant form eg Wikipedia:Good_article_nominations#Transport  "'

In [14]:
# проверка текст до
df['lemm_text'][3]

"More I ca n't make any real suggestion on improvement I wonder if the section statistic should be later on or a subsection of type of accident I think the reference may need tidy so that they be all in the exact same format ie date format etc I can do that later on if no one else do first if you have any preference for format style on reference or want to do it yourself please let me know There appear to be a backlog on article for review so I guess there may be a delay until a reviewer turn up It 's list in the relevant form eg Wikipedia Good article nomination Transport"

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 4 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
 3   lemm_text   159292 non-null  object
dtypes: int64(2), object(2)
memory usage: 4.9+ MB


### Разделение выборок

In [16]:
X=df.drop('toxic',axis=1)
y=df['toxic']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42, shuffle=True,stratify=y)

In [18]:
# проверка
train_sample=X_train.shape[0]/X.shape[0]
test_sample=X_test.shape[0]/X.shape[0]

print('Размер тренировочной выборки- {:.0%}'.format(train_sample))
print('Размер тестовой выборки - {:.0%}'.format(test_sample))

Размер тренировочной выборки- 90%
Размер тестовой выборки - 10%


### Используем технологию TF-IDF для обработки данныx, а так же отфильтруем содержимое при помощи библиотеки nltk

In [19]:
# Загрузим библиотеку и стоп-слова из английского языка
nltk.download('stopwords') 
stop_words = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
# Создадим корпус из лемматизированных и очищенных слов 
corpus_lemm_train =X_train['lemm_text']
corpus_lemm_train.shape

(143362,)

In [21]:
# Применим модель TfidfVectorizer и обучим её на наших тренировочных данных
count_tf_idf = TfidfVectorizer(stop_words=stop_words,analyzer='word') 
tf_idf_train = count_tf_idf.fit_transform(corpus_lemm_train) 
tf_idf_train.shape

(143362, 150065)

In [22]:
# Создадим так же корпус слов и для тестовой выборки
corpus_lemm_test = X_test['lemm_text']
corpus_lemm_test.shape

(15930,)

In [23]:
# Трансформируем тестовую выборку
tf_idf_test = count_tf_idf.transform(corpus_lemm_test)
tf_idf_test.shape

(15930, 150065)

## Обучение

Обучим модели и оценим их качество кросс-валидацией. Для решения задачи подойдут 3 модели: логистическая регрессия, решающее дерево и случайный лес. Будем замерять метрики качества F1.Начнем с логистической регрессии.

### Логистическая регрессия

In [25]:
# Попробуем подобрать параметры для 
# Логистическая регрессия
par_grid_logist = {
                   'class_weight': [None, 'balanced'],
                   'C': [5, 10, 15]
                   }
model_lr = LogisticRegression(solver='liblinear',random_state=12345)

grid_search_lr = GridSearchCV(model_lr, par_grid_logist, cv=3,
                           scoring='f1')
grid_search_lr.fit(tf_idf_train, y_train)

GridSearchCV(cv=3,
             estimator=LogisticRegression(random_state=12345,
                                          solver='liblinear'),
             param_grid={'C': [5, 10, 15], 'class_weight': [None, 'balanced']},
             scoring='f1')

In [26]:
grid_search_lr.best_params_

{'C': 10, 'class_weight': None}

In [27]:
print(grid_search_lr.best_score_)

0.7594941723961804


In [28]:
grid_search_lr.cv_results_

{'mean_fit_time': array([12.83832955, 21.94621412, 14.74489156, 25.17791041, 17.61340038,
        29.31140979]),
 'std_fit_time': array([0.49996373, 0.20563252, 0.34074264, 0.57015827, 0.34060735,
        2.11417435]),
 'mean_score_time': array([0.0174849 , 0.01805242, 0.01737396, 0.01807149, 0.0176932 ,
        0.01814985]),
 'std_score_time': array([6.70367586e-05, 3.66921847e-04, 4.65690113e-04, 2.28423774e-04,
        1.30499324e-04, 4.70916388e-04]),
 'param_C': masked_array(data=[5, 5, 10, 10, 15, 15],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_class_weight': masked_array(data=[None, 'balanced', None, 'balanced', None, 'balanced'],
              mask=[False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 5, 'class_weight': None},
  {'C': 5, 'class_weight': 'balanced'},
  {'C': 10, 'class_weight': None},
  {'C': 10, 'class_weight': 'balanced'},

In [29]:
print(grid_search_lr.best_estimator_)

LogisticRegression(C=10, random_state=12345, solver='liblinear')


In [30]:
pd.DataFrame(grid_search_lr.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_class_weight,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,12.838330,0.499964,0.017485,0.000067,5,None,"{'C': 5, 'class_weight': None}",0.761009,0.754181,0.747769,0.754320,0.005406,6
1,21.946214,0.205633,0.018052,0.000367,5,balanced,"{'C': 5, 'class_weight': 'balanced'}",0.754710,0.758991,0.754811,0.756171,0.001995,4
2,14.744892,0.340743,0.017374,0.000466,10,None,"{'C': 10, 'class_weight': None}",0.765449,0.759001,0.754032,0.759494,0.004674,1
3,25.177910,0.570158,0.018071,0.000228,10,balanced,"{'C': 10, 'class_weight': 'balanced'}",0.755265,0.761152,0.756074,0.757497,0.002605,3
4,17.613400,0.340607,0.017693,0.000130,15,None,"{'C': 15, 'class_weight': None}",0.763611,0.756813,0.756121,0.758848,0.003380,2
5,29.311410,2.114174,0.018150,0.000471,15,balanced,"{'C': 15, 'class_weight': 'balanced'}",0.752728,0.759225,0.753362,0.755105,0.002924,5


Теперь перейдем к модели решающего дерева. Переберем гиперпараметры (размер глубины), и выведем на экран при какой глубине результат кросс-валидации дал наивысшую оценку качества модели:

### Модель дерево решений

In [31]:
# Модель дерево решений
par_grid_DT = {
                     'criterion': ['gini', 'entropy'],
                     'max_depth': np.arange(1, 10)
                      
              }
model_grid_DT = DecisionTreeClassifier(random_state=12345)

grid_search_dt = GridSearchCV(model_grid_DT, par_grid_DT, cv=3,
                           scoring= 'f1'
                          )
grid_search_dt.fit(tf_idf_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=12345),
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': array([1, 2, 3, 4, 5, 6, 7, 8, 9])},
             scoring='f1')

In [32]:
grid_search_dt.best_params_

{'criterion': 'gini', 'max_depth': 9}

In [33]:
print(grid_search_dt.best_score_)

0.5812529193938835


In [34]:
grid_search_dt.cv_results_

{'mean_fit_time': array([ 6.50625269,  6.90441934,  7.31804307,  7.68946171,  8.09398007,
         8.52912219,  8.96523436,  9.46449518,  9.91690691,  6.66880902,
         7.11144646,  7.57597033,  8.1310753 ,  8.61738801,  9.1351908 ,
         9.72348992, 10.34767834, 10.94368378]),
 'std_fit_time': array([0.04637266, 0.05074503, 0.01265333, 0.03281198, 0.04320892,
        0.02784395, 0.05298462, 0.03377721, 0.09087437, 0.07796782,
        0.0549464 , 0.02130795, 0.04783796, 0.07697766, 0.03088672,
        0.06065346, 0.01053283, 0.05904264]),
 'mean_score_time': array([0.057     , 0.05680863, 0.05898809, 0.05749528, 0.0567619 ,
        0.05781444, 0.05710983, 0.05687221, 0.0573229 , 0.0568099 ,
        0.05612032, 0.05834707, 0.05706271, 0.05728817, 0.05701605,
        0.05820934, 0.05712668, 0.05706986]),
 'std_score_time': array([0.00078183, 0.00057659, 0.00192148, 0.00054865, 0.0004072 ,
        0.00025835, 0.00024912, 0.00091261, 0.00045298, 0.00048394,
        0.00055929, 0.0023

In [35]:
print(grid_search_dt.best_estimator_)

DecisionTreeClassifier(max_depth=9, random_state=12345)


In [36]:
pd.DataFrame(grid_search_dt.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,6.506253,0.046373,0.057000,0.000782,gini,1,"{'criterion': 'gini', 'max_depth': 1}",0.352981,0.349528,0.356266,0.352925,0.002751,17
1,6.904419,0.050745,0.056809,0.000577,gini,2,"{'criterion': 'gini', 'max_depth': 2}",0.410875,0.410455,0.416921,0.412750,0.002954,15
2,7.318043,0.012653,0.058988,0.001921,gini,3,"{'criterion': 'gini', 'max_depth': 3}",0.460941,0.453792,0.459716,0.458150,0.003122,14
3,7.689462,0.032812,0.057495,0.000549,gini,4,"{'criterion': 'gini', 'max_depth': 4}",0.495338,0.495094,0.493902,0.494778,0.000628,12
4,8.093980,0.043209,0.056762,0.000407,gini,5,"{'criterion': 'gini', 'max_depth': 5}",0.516091,0.516657,0.522819,0.518522,0.003047,10
5,8.529122,0.027844,0.057814,0.000258,gini,6,"{'criterion': 'gini', 'max_depth': 6}",0.545481,0.544177,0.540870,0.543509,0.001941,6
6,8.965234,0.052985,0.057110,0.000249,gini,7,"{'criterion': 'gini', 'max_depth': 7}",0.550541,0.557986,0.556186,0.554904,0.003172,4
7,9.464495,0.033777,0.056872,0.000913,gini,8,"{'criterion': 'gini', 'max_depth': 8}",0.560986,0.574845,0.569298,0.568376,0.005695,2
8,9.916907,0.090874,0.057323,0.000453,gini,9,"{'criterion': 'gini', 'max_depth': 9}",0.581994,0.584453,0.577311,0.581253,0.002963,1
9,6.668809,0.077968,0.056810,0.000484,entropy,1,"{'criterion': 'entropy', 'max_depth': 1}",0.352981,0.349528,0.356266,0.352925,0.002751,17


Теперь перейдем к модели случайного леса. Аналогично работе с предыдущей моделью, в цикле переберем значения гиперпараметра n_estimators, который отвечает за количество деревьев в лесу. Выведем на экран значение n_estimators при котором возможно получить максимальную оценку качества модели, полученную кросс-валидацией:

### Модель случайный лес

In [37]:
# Модель случайный лес
par_grid_ensemble = {'n_estimators': [3, 10],
                     'criterion': ['gini', 'entropy'],
                     'min_samples_split': range(5, 10)
                    }
model_RF = RandomForestClassifier(random_state=12345)

grid_search_rf = GridSearchCV(model_RF, par_grid_ensemble, cv=3,
                           scoring='f1'
                          )
grid_search_rf.fit(tf_idf_train, y_train)

GridSearchCV(cv=3, estimator=RandomForestClassifier(random_state=12345),
             param_grid={'criterion': ['gini', 'entropy'],
                         'min_samples_split': range(5, 10),
                         'n_estimators': [3, 10]},
             scoring='f1')

In [38]:
grid_search_rf.best_params_

{'criterion': 'gini', 'min_samples_split': 8, 'n_estimators': 10}

In [39]:
print(grid_search_rf.best_score_)

0.6638024473045809


In [40]:
grid_search_rf.cv_results_

{'mean_fit_time': array([21.71304393, 72.73820488, 21.65727568, 70.78069186, 20.75214299,
        68.59166137, 20.59662429, 66.80746341, 20.14525803, 66.03667482,
        19.38654542, 63.80862323, 18.78732387, 62.00248305, 18.3540446 ,
        60.82910482, 18.13548676, 59.71142451, 17.89901598, 58.68428453]),
 'std_fit_time': array([0.47834635, 1.21423739, 0.78823746, 0.86910907, 0.59695487,
        0.60045393, 0.68267321, 0.63807969, 0.61228968, 0.54082115,
        0.28341983, 0.51976985, 0.25357687, 0.62787019, 0.15292787,
        0.23892444, 0.24710571, 0.57431054, 0.12711081, 0.43142417]),
 'mean_score_time': array([0.28186568, 0.80291136, 0.28266668, 0.80013331, 0.27813085,
        0.80179207, 0.28024888, 0.80502208, 0.28054301, 0.80172356,
        0.2253716 , 0.63828317, 0.22489913, 0.63658897, 0.22710101,
        0.63602567, 0.22881929, 0.6374774 , 0.22587156, 0.63370268]),
 'std_score_time': array([0.00977433, 0.0098875 , 0.00917208, 0.00792085, 0.00633618,
        0.00965961, 

In [41]:
print(grid_search_rf.best_estimator_)

RandomForestClassifier(min_samples_split=8, n_estimators=10, random_state=12345)


In [42]:
pd.DataFrame(grid_search_rf.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_criterion,param_min_samples_split,param_n_estimators,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,21.713044,0.478346,0.281866,0.009774,gini,5,3,"{'criterion': 'gini', 'min_samples_split': 5, ...",0.641099,0.633279,0.624148,0.632842,0.006927,8
1,72.738205,1.214237,0.802911,0.009888,gini,5,10,"{'criterion': 'gini', 'min_samples_split': 5, ...",0.665011,0.664660,0.658935,0.662869,0.002785,2
2,21.657276,0.788237,0.282667,0.009172,gini,6,3,"{'criterion': 'gini', 'min_samples_split': 6, ...",0.639930,0.626942,0.629743,0.632205,0.005581,10
3,70.780692,0.869109,0.800133,0.007921,gini,6,10,"{'criterion': 'gini', 'min_samples_split': 6, ...",0.668323,0.655427,0.653841,0.659197,0.006485,4
4,20.752143,0.596955,0.278131,0.006336,gini,7,3,"{'criterion': 'gini', 'min_samples_split': 7, ...",0.640271,0.636825,0.620822,0.632639,0.008474,9
5,68.591661,0.600454,0.801792,0.009660,gini,7,10,"{'criterion': 'gini', 'min_samples_split': 7, ...",0.664745,0.666404,0.654111,0.661754,0.005446,3
6,20.596624,0.682673,0.280249,0.008033,gini,8,3,"{'criterion': 'gini', 'min_samples_split': 8, ...",0.647486,0.630951,0.636800,0.638412,0.006846,6
7,66.807463,0.638080,0.805022,0.014389,gini,8,10,"{'criterion': 'gini', 'min_samples_split': 8, ...",0.668316,0.659234,0.663858,0.663802,0.003708,1
8,20.145258,0.612290,0.280543,0.007678,gini,9,3,"{'criterion': 'gini', 'min_samples_split': 9, ...",0.637238,0.632710,0.635159,0.635036,0.001851,7
9,66.036675,0.540821,0.801724,0.005080,gini,9,10,"{'criterion': 'gini', 'min_samples_split': 9, ...",0.671104,0.655467,0.648222,0.658264,0.009549,5


In [47]:
print('Логистическая регрессия:', 0.76)
print('Модель дерево решений:', 0.58)
print('Модель случайный лес:', 0.66)

Логистическая регрессия: 0.76
Модель дерево решений: 0.58
Модель случайный лес: 0.66


###### ВЫВОД: 
- Модель логистическая регрессия: 0.76 лучшая.Теперь проверим ее на тестовой выборке. 

In [45]:
predicted = grid_search_lr.predict(tf_idf_test)
f1 = f1_score(y_test, predicted)

print("f1:", f1)

f1: 0.7863013698630138


ВЫВОД: Модель LogisticRegression показала результат на тесте метрики F1= 0.78.

## Выводы

На первом этапе произведена загрузка данных и их подготовка для обучения моделей. 
Для анализа данных и построения модели предоставлен датасет с размеченными данными, содержащий комментарии пользователей к товарам, доступным для приобретения в интернет-магазине «Викишоп». Датасет состоит из 3 столбцов с данными и 159292 строк. 
Затем тексты комментариев были очищены и лемматизированы. 
Проверка соотношения классов показала, что в датасете имеет место явный дисбаланс. Мажорный класс составляет лишь 10% от всего датасета. 
После векторизации датасет образовал матрицу размером (119469, 134385). В итоге полученный датасет разделен на обучающую 90 % и тестовые выборки 10 %.

В качестве моделей использованы Логистическая регрессия, Дерево решений, Случайный лес. 
По результатам подбора гиперпараметров и обучения моделей наилучший результат показала логистическая регрессия, наихудший - Дерево решений. 
В связи с этим модель Логистической регрессии подходит для решения поставленной задачи.Модель LogisticRegression на тесте показала значение метрики F1= 0.78.